# 01 - Metadata Parser (OrbitCheck)

This notebook creates **standardized metadata JSON files** for images used in OrbitCheck.

Output:
- Image saved to: `data/raw/images/`
- Metadata saved to: `data/raw/metadata/<image_id>.json`

This standard metadata format is used by all later notebooks.


In [ ]:
import os, json, shutil
from datetime import datetime, timezone
from google.colab import files

PROJECT_ROOT = "/content/orbitcheck"
IMG_DIR = os.path.join(PROJECT_ROOT, "data/raw/images")
META_DIR = os.path.join(PROJECT_ROOT, "data/raw/metadata")

os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(META_DIR, exist_ok=True)

print("IMG_DIR:", IMG_DIR)
print("META_DIR:", META_DIR)


In [ ]:
def build_standard_metadata(
    image_id: str,
    image_filename: str,
    timestamp_utc: str,
    lat: float,
    lon: float,
    source: str = "manual",
    sun_azimuth=None,
    sun_elevation=None,
    tle_line1=None,
    tle_line2=None,
    footprint=None
):
    """
    Standard metadata schema for OrbitCheck.
    """
    # Validate timestamp format
    dt = datetime.strptime(timestamp_utc, "%Y-%m-%d %H:%M:%S").replace(tzinfo=timezone.utc)

    meta = {
        "image_id": image_id,
        "image_filename": image_filename,
        "timestamp_utc": dt.strftime("%Y-%m-%d %H:%M:%S"),
        "location": {
            "lat": float(lat),
            "lon": float(lon)
        },
        "source": source,
        "sun_metadata": {
            "azimuth": sun_azimuth,
            "elevation": sun_elevation
        },
        "orbit": {
            "tle_line1": tle_line1,
            "tle_line2": tle_line2
        },
        "footprint": footprint  # optional GeoJSON polygon
    }
    return meta


In [ ]:
print("Upload a satellite/overhead image (JPG/PNG recommended):")
uploaded = files.upload()

if len(uploaded) == 0:
    raise ValueError("No image uploaded.")

image_name = list(uploaded.keys())[0]
src_path = f"/content/{image_name}"
dst_path = os.path.join(IMG_DIR, image_name)

shutil.move(src_path, dst_path)
print("Image stored:", dst_path)


In [ ]:
print("\nEnter metadata for this image:")

image_id = input("Image ID (example: sample1): ").strip()
timestamp_utc = input("UTC timestamp (YYYY-MM-DD HH:MM:SS): ").strip()
lat = float(input("Latitude: "))
lon = float(input("Longitude: "))
source = input("Source (google_earth / sentinel2 / landsat / manual): ").strip()

meta = build_standard_metadata(
    image_id=image_id,
    image_filename=image_name,
    timestamp_utc=timestamp_utc,
    lat=lat,
    lon=lon,
    source=source
)

meta_path = os.path.join(META_DIR, f"{image_id}.json")
with open(meta_path, "w") as f:
    json.dump(meta, f, indent=2)

print("\n Metadata saved:", meta_path)
print(json.dumps(meta, indent=2))


In [ ]:
print("\n Images in IMG_DIR:")
print(os.listdir(IMG_DIR))

print("\n Metadata JSON files in META_DIR:")
print(os.listdir(META_DIR))
